In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["kin1"]

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Numerical solution of chemical equilibrium problems #3
Try now to determine the amount of acetic acid and acetate in a solution whose pH is kept constat at a chosen value by external titration. Assume that this is the only reaction taking plane in the system
\begin{equation}
\mathrm{CH_3COOH \rightleftharpoons CH_3COO^{-} + H^+} \qquad pK_{a} = 3.74
\end{equation}


- Click `Download CSV` to export the data as a CSV file to verify your result.


In [3]:
def initialise():
    global nPoints
    global concA, concB, pH
    global pKeq1
    global delta

    nPoints = 500
    concA = 0.1
    concB = 0.1
    pH = 7
    pKeq1 = 4.76
    delta = 0.1

def addLine(t,x,y,q1,res):
    var_list = []
    var_list.append(t)
    var_list.append(x)
    var_list.append(y)
    var_list.append(q1)
    res.loc[len(res)] = var_list

initialise()


In [5]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

    
def force(Q,pk):
    if (Q > 0):
        force = - (math.log(Q,10) + pk)
    else:
        force = 1.

    return force

def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(respath):
        os.remove(respath)
    res = pd.DataFrame(columns=["step" 
                                , "CH$_3$COOH" 
                                , "CH$_3$COO$^{-}$" 
                                , "p($Q_1$)"
                               ])

    A  = float(concA_text.value)
    B  = float(concB_text.value)
    H  = math.pow(10,-float(pH_text.value))

    dx = float(delta_text.value)
    pk1 = float(pKeq1_text.value)
    n  = int(nPoints_text.value)

    Q1 = B * H / A
    
    addLine(1,A,B,
            -math.log(Q1,10),
            res)
    
    for i in range(0, n):
        f1 = force(Q1,pk1)

        dx1 = min(A,B)*(dx * f1)
        
        A = A - dx1
        B = B + dx1

        Q1 = B * H / A            

        addLine(i+1,A,B,
                -math.log(Q1,10),
                res)

    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(n))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equilibrium constant
pKeq1_text = ipw.Text(str(pKeq1))

# Initial concentrations
concA_text = ipw.Text(str(concA))
concB_text = ipw.Text(str(concB))
pH_text = ipw.Text(str(pH))

# delta concentration
delta_text = ipw.Text(str(delta))

# Nmber of data points
nPoints_text = ipw.Text(str(nPoints))

rows.append(ipw.HBox([ipw.Label('Initial concentration of "CH$_3$COOH" :  '),concA_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of "CH$_3$OO$^{-}$" :  '),concB_text]))
rows.append(ipw.HBox([ipw.Label('Buffer pH    :  '),pH_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($pK_1$)          :  '),pKeq1_text]))
rows.append(ipw.HBox([ipw.Label('Delta concentration                   :  '),delta_text]))
rows.append(ipw.HBox([ipw.Label('Number of data point required         :  '),nPoints_text]))

rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)